# SMA Project

In [2]:
# load libraries
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tweepy
import itertools
import matplotlib.pyplot as plt
import networkx as nx

#### Data Collection

In [2]:
api_key = 'Aj8dar7L332QDBvptAp8mVtKz'
secret_key = 'mGjaEJFk8EL4DaBiNp5TN4Zmi5d7MPPCL3g1lwPZQGk8v3d64D'
access_token = '2397096199-0LXa1ze00wodZvqcgAh4Nvro3yKqyyHSWpIEnwI'
access_token_secret = 'T5j56DBAxO3FEYoUex780YEdb4MC5dVnTu9cF9TpLWT3s'

auth = tweepy.OAuthHandler(api_key,secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit = True)

In [3]:
# get tweets based on search query
cols = ['type','query','datetime','A_handle','A_follower_count','A_following_count','A_listed_count','A_posts',
        'B_handle','B_follower_count','B_following_count','B_listed_count','B_posts','text']
tweets = pd.DataFrame(columns=cols)
tweets_raw = []
text_query = '#ercot' # '#dogecoin'
queries = ['#texas','#texasblackout','#texasfreeze','#ercot','#texasdisaster']
queries = ['#texaspoweroutage','#texaswinterstorm2021']
tweets = pd.read_csv('tweets_texas_dates2.csv')

for text_query in queries:
    count = np.where(text_query == '#texasfreeze',3000,20000)
    # Pulling individual tweets from query
    for tweet in tweepy.Cursor(api.search,
                               q=text_query,
                               count=100, # number of results per page
                               result_type="recent",
                               include_entities=True).items(count): # total number of results
        try:
            tweets_raw.append(tweet)
            # Adding to list that contains all tweets    
            handle = tweet._json['user']['screen_name']
            if tweet.text[:2] == 'RT': # they are retweeting
                rt_handle = tweet._json['retweeted_status']['user']['screen_name']
                rt_followers = tweet._json['retweeted_status']['user']['followers_count']
                rt_following = tweet._json['retweeted_status']['user']['friends_count']
                rt_listed = tweet._json['retweeted_status']['user']['listed_count']
                rt_posts = tweet._json['retweeted_status']['user']['statuses_count']
                content = 'RT'
            elif False:
                # they are quoting a tweet
                pass
            else:
                rt_handle = handle
                content = 'Tweet'
            followers = tweet._json['user']['followers_count']
            following = tweet._json['user']['friends_count']
            listed = tweet._json['user']['listed_count']
            posts = tweet._json['user']['statuses_count']
            text = tweet._json['text']
            datetime = tweet._json['created_at']

            tweets = tweets.append(pd.Series({'type':content,'query':text_query,'datetime':datetime,'A_handle':handle,
                                              'A_follower_count':followers,'A_following_count':following,'A_listed_count':listed,
                                              'A_posts':posts,'B_handle':rt_handle,'B_follower_count':rt_followers,
                                              'B_following_count':rt_following,'B_listed_count':rt_listed,
                                              'B_posts':rt_posts,'text':text}), ignore_index = True)
        except:
            continue
    
    pd.DataFrame(tweets).to_csv('tweets_texas_dates3.csv',index=False)
    print(text_query, 'completed')

#texaspoweroutage completed
#texaswinterstorm2021 completed


In [4]:
pd.DataFrame(tweets).to_csv('tweets_'+text_query.strip('#')+'.csv',index=False)

In [3]:
tweets = pd.read_csv('tweets_texas_dates3.csv')

In [4]:
print(tweets.shape)
tweets.groupby('query')['type'].count()

(26712, 14)


query
#ercot                    2701
#texasblackout           13512
#texasdisaster              25
#texasfreeze              3000
#texaspoweroutage         3256
#texaswinterstorm2021     4218
Name: type, dtype: int64

In [20]:
dates = tweets['datetime'].map(lambda x: str(x.split()[1] + ' ' + x.split()[2]))
dates = pd.to_datetime(tweets['datetime'], errors='coerce')

In [21]:
dates.max(), dates.min()

(Timestamp('2021-03-02 18:04:26+0000', tz='UTC'),
 Timestamp('2021-02-22 08:22:59+0000', tz='UTC'))

In [22]:
tweets.columns

Index(['type', 'query', 'datetime', 'A_handle', 'A_follower_count',
       'A_following_count', 'A_listed_count', 'A_posts', 'B_handle',
       'B_follower_count', 'B_following_count', 'B_listed_count', 'B_posts',
       'text'],
      dtype='object')

Notes:
- #Dogecoin pulled 2/23/2021 8:24 pm (4998)
- #Texas pulled 2/23/2021 10:40 pm (4998)
- #TexasBlackout pulled 2/23/2021 10:41 pm (5000)
- #Texasfreeze pulled 2/23/2021 11:15 pm (2999)
- #ERCOT pulled 2/24/2021 12:34 pm (4999)
- Repulled texas related hashtags + #texasdisaster 2/24/2021 1:11 pm w/ dates (18159)
- Repulled texas related hashtags 3/1/2021 4:04 pm (39232)
- Added #texaspoweroutage and #texaswinterstorm2021 and removed #texas on 3/2/2021 12:28 pm (46706)

#### Generate and Visualize Graph

In [21]:
df = pd.read_csv('tweets_texas_dates.csv')

In [22]:
Graph = nx.from_pandas_edgelist(df,source='B_handle',target='A_handle', edge_attr=None, create_using=nx.DiGraph())
print(nx.info(Graph))

Name: 
Type: DiGraph
Number of nodes: 15226
Number of edges: 15691
Average in degree:   1.0305
Average out degree:   1.0305


In [71]:
plt.savefig('graph_'+text_query.strip('#')+'.png') 

<Figure size 432x288 with 0 Axes>

#### Include Centrality Metrics

In [23]:
betweenness = nx.betweenness_centrality(Graph)
degree = nx.degree_centrality(Graph)
out_degree = nx.out_degree_centrality(Graph)
in_degree = nx.in_degree_centrality(Graph)
closeness = nx.closeness_centrality(Graph)

In [24]:
bet_df = pd.DataFrame.from_dict(betweenness, orient='index', columns=['Betweenness'])
degree_df = pd.DataFrame.from_dict(degree, orient='index', columns=['Degree'])
close_df = pd.DataFrame.from_dict(closeness, orient='index', columns=['Closeness'])
indegree_df = pd.DataFrame.from_dict(in_degree, orient='index', columns=['In Degree'])
outdegree_df = pd.DataFrame.from_dict(out_degree, orient='index', columns=['Out Degree'])

In [25]:
metrics = bet_df.merge(degree_df, left_index=True, right_index = True)
metrics = metrics.merge(close_df, left_index = True, right_index = True)
metrics = metrics.merge(indegree_df, left_index = True, right_index = True)
metrics = metrics.merge(outdegree_df, left_index = True, right_index = True)
metrics.head()

,Betweenness,Degree,Closeness,In Degree,Out Degree
themattwalton,0.0,0.240394,0.000000,0.000000,0.240394
jakreyno,0.0,0.000066,0.000066,0.000066,0.000000
CathynotKathy,0.0,0.000066,0.000066,0.000066,0.000000
scribler2,0.0,0.000066,0.000066,0.000066,0.000000
SusanOrdway,0.0,0.000066,0.000066,0.000066,0.000000


In [55]:
t = pd.Series(metrics['Betweenness'].unique())
t = list(t.sort_values()[:-20])
metrics['Betweenness'].isin(t).value_counts()

True     15201
False       25
Name: Betweenness, dtype: int64

In [26]:
merge = df.merge(metrics, how='left',left_on='A_handle',right_index=True)
merge = merge.merge(metrics, how='left',left_on='B_handle',right_index=True,suffixes=['_A','_B'])
merge #= merge.drop(columns=[''])

,type,query,datetime,A_handle,A_follower_count,A_following_count,A_listed_count,A_posts,B_handle,B_follower_count,...,Betweenness_A,Degree_A,Closeness_A,In Degree_A,Out Degree_A,Betweenness_B,Degree_B,Closeness_B,In Degree_B,Out Degree_B
0,RT,#texas,Wed Feb 24 19:12:20 +0000 2021,jakreyno,8,282,0,103,themattwalton,6520,...,0.0,0.000066,0.000066,0.000066,0.000000,0.0,0.240394,0.0,0.000000,0.240394
1,RT,#texas,Wed Feb 24 19:12:18 +0000 2021,CathynotKathy,32,130,0,67,themattwalton,6520,...,0.0,0.000066,0.000066,0.000066,0.000000,0.0,0.240394,0.0,0.000000,0.240394
2,RT,#texas,Wed Feb 24 19:12:16 +0000 2021,scribler2,1112,1279,4,153287,themattwalton,6520,...,0.0,0.000066,0.000066,0.000066,0.000000,0.0,0.240394,0.0,0.000000,0.240394
3,RT,#texas,Wed Feb 24 19:12:15 +0000 2021,SusanOrdway,243,797,17,4360,themattwalton,6520,...,0.0,0.000066,0.000066,0.000066,0.000000,0.0,0.240394,0.0,0.000000,0.240394
4,RT,#texas,Wed Feb 24 19:12:13 +0000 2021,JuliaIMorini,206,1334,0,5554,themattwalton,6520,...,0.0,0.000066,0.000066,0.000066,0.000000,0.0,0.240394,0.0,0.000000,0.240394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18154,Tweet,#texasdisaster,Wed Feb 17 18:38:48 +0000 2021,nmeah,38,144,6,620,nmeah,62229,...,0.0,0.000131,0.000000,0.000066,0.000066,0.0,0.000131,0.0,0.000066,0.000066
18155,Tweet,#texasdisaster,Wed Feb 17 17:29:53 +0000 2021,nmeah,38,144,6,620,nmeah,62229,...,0.0,0.000131,0.000000,0.000066,0.000066,0.0,0.000131,0.0,0.000066,0.000066
18156,Tweet,#texasdisaster,Wed Feb 17 15:01:17 +0000 2021,arbyoliver,226,47,24,15572,arbyoliver,62229,...,0.0,0.000131,0.000000,0.000066,0.000066,0.0,0.000131,0.0,0.000066,0.000066
18157,Tweet,#texasdisaster,Wed Feb 17 12:39:24 +0000 2021,bradengw,1770,4382,276,75938,bradengw,62229,...,0.0,0.000131,0.000000,0.000066,0.000066,0.0,0.000131,0.0,0.000066,0.000066


#### Incoperate Part 1 Important Metrics

In [8]:
# Code Mapping from part 1: nw_feat_1:degree, nw_feat_2:Betweenness, nw_feat_3:Closeness
# Important Features from Best Model (XGBoost): 
# 1. A-B_listed_count
# 2. A-B_follower_count
# 3. A-B_following_count
# 4. A-B_mentions_sent
# 5. A-B_betweenness
# 6. A-B_posts
# 7. A-B_retweets_sent
# 8. A-B_closeness

cols = ['type','A_handle','A_follower_count','A_following_count','A_listed_count','A_posts',
        'Betweenness_A','Degree_A','Closeness_A','In Degree_A','Out Degree_A',
        'B_handle','B_follower_count','B_following_count','B_listed_count','B_posts',
        'Betweenness_B','Degree_B','Closeness_B','In Degree_B','Out Degree_B']
merge = merge[cols]
temp = pd.DataFrame(merge.iloc[:,2:11].values - merge.iloc[:,12:21].values)
temp = pd.DataFrame(StandardScaler().fit_transform(temp))
temp.columns = ['A-B_follower_count','A-B_following_count','A-B_listed_count','A-B_mentions_sent',
                'A-B_retweets_sent','A-B_posts','A-B_degree','A-B_betweenness','A-B_closeness']
m = merge.merge(temp, left_index=True, right_index=True)
m.head()

,type,A_handle,A_follower_count,A_following_count,A_listed_count,A_posts,Betweenness_A,Degree_A,Closeness_A,In Degree_A,...,Out Degree_B,A-B_follower_count,A-B_following_count,A-B_listed_count,A-B_mentions_sent,A-B_retweets_sent,A-B_posts,A-B_degree,A-B_betweenness,A-B_closeness
0,RT,AdalhelmLP,33,15,9,2639,0.0,0.00024,0.00024,0.00024,...,0.000721,0.237144,0.076780,0.265712,0.029769,0.082259,0.341186,0.103557,-0.214311,0.361968
1,RT,FizzOfInsanity,76,636,5,6275,0.0,0.00024,0.00024,0.00024,...,0.004804,-0.061599,-0.010354,-0.311025,-0.389910,0.082259,-0.423068,0.103557,0.340363,-0.455304
2,Tweet,angler_mx,1523,1768,106,18155,0.0,0.00048,0.00000,0.00024,...,0.000240,-0.059378,0.065345,-0.283587,-0.338467,0.082259,0.484484,-0.417975,-0.214311,0.506192
3,RT,luisjav44152468,422,827,2,68424,0.0,0.00048,0.00048,0.00048,...,0.014172,0.236903,0.037527,0.262996,-0.161864,-8.227367,-2.333702,0.103557,-0.214311,-2.330222
4,Tweet,chai_tx,9,29,0,2156,0.0,0.00048,0.00000,0.00024,...,0.000240,0.236269,-0.015837,0.262452,-0.448818,0.082259,0.484484,-0.417975,-0.214311,0.506192


#### Score and Analyize

In [64]:
# add score
m['score'] = .1*m['A-B_listed_count'] + .1*m['A-B_follower_count'] + .1*m['A-B_following_count'] + \
             .1*m['A-B_mentions_sent'] + .1*m['A-B_betweenness'] + .1*m['A-B_posts'] + \
             .1*m['A-B_retweets_sent'] + .1*m['A-B_closeness']

In [69]:
m_sorted = m.sort_values(by='score', ascending=False)[['A_handle','score']]
top = m_sorted.groupby('A_handle').sum()
top = top.sort_values(by='score', ascending=False)
top[0:25]

,score
A_handle,
YouCanFoolMost,34.375879
Ljr16015824,16.545004
Gambiste1,14.939121
MarcoDaCostaFX,14.602559
RT_com,8.981116
AppleRTweet,6.161499
ExpansionMx,4.118235
peruenlanoticia,3.587478
smallcappick,3.439288
